In [1]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
import time
from tqdm.contrib.telegram import tqdm
from transliterate import to_latin
import json
import os

In [2]:
tqdm.pandas()

In [3]:
TOKEN = os.getenv("TOKEN")
CHAT_ID = os.getenv("CHAT_ID")

In [4]:
BASE_URL = "https://islom.uz"

In [5]:
main_link = "https://islom.uz/maqolalar/6/1"

In [6]:
links = []

In [7]:
async def main():
    async with aiohttp.ClientSession() as session:
        
        for menu in tqdm(navbar_links, token=TOKEN, chat_id=CHAT_ID):
            for page in tqdm(range(2500), token=TOKEN, chat_id=CHAT_ID):
                try:   
                    async with session.get(menu.format(page=page)) as resp:
                        if resp.status == 200:
                            data = await resp.text()
                            soup = bs4.BeautifulSoup(data)
                            selected_links = soup.find('div', class_='col-7 in_block category_body').find_all('div', class_="col-6 smilar_state")
                            links.extend([BASE_URL+link.find('a')['href'] for link in selected_links])
                        elif resp.status == 500:
                            break
                        elif resp.status != 404:
                            await asyncio.sleep(60) # 1 minute

                except Exception as e:
                    print(e, "page_num", page)


In [ ]:
await main()

In [ ]:
results = []

async def main():
    async with aiohttp.ClientSession() as session:

        for link in tqdm(links, token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(link) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        
                        main_block = soup.find('div', class_="col-7 in_block category_body")
                        
                        title = main_block.find('h3', class_="title_inmaqola").text

                        content = main_block.find('div', class_="inmaqola_text").text
                        
                        results.append({
                            "title": title.strip(),
                            "content": content.strip()
                        })
                    elif resp.status != 404:
                        await asyncio.sleep(60)
            except Exception as e:
                print(e, link)
    
await main()

In [ ]:
pd.DataFrame(results).to_pickle("islomuz")

# Filtering text

In [8]:
def filter_text(text):
    texts = text.split(".")
    
    for i, t in enumerate(texts):
        for char in t:
            if ord(char)> 122:
                texts[i] = ""
                break
        
    
    texts = [text for text in texts if text != ""]
    
    return ".".join(texts)

In [9]:
def replace_values(text):
    replaces = {"ʹ" : "'", "ʺ" : '"', "ʻ" : "'", "ʼ" : "'", "ʽ" : "'", 'ˮ' : '"',  "`":"'", "‘":"'", 
                "’":"'", "‚":",", "‛":"'", '“':'"', '”':'"', '“':'"', '„':',', '‟': '"', "․": ".", "‥":'.', "…":".", "′":"'",
               "″":'"', '‴':'"', "«":'"', "»":'"', "ʼ":"'", "ʻ":"'", "\xa0":""}
    
    for i,j in replaces.items():
        text = text.replace(i, j)
    return text

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df2 = df.copy()

In [ ]:
df2['title'] = df2['title'].progress_apply(lambda x: to_latin(x))

df2['content'] = df2['content'].progress_apply(lambda x: to_latin(x))

In [ ]:
df3 = df2.copy()

In [ ]:
df3['content'] = df3['content'].progress_apply(lambda x: replace_values(x))

In [ ]:
df3['content'] = df3['content'].progress_apply(lambda x: filter_text(x))

In [ ]:
df3.to_pickle('islomuz2.pickle')